In [1]:
%load_ext autoreload
%autoreload 1

#  https://colab.research.google.com/notebooks/data_table.ipynb

from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive
drive.mount('/content/gdrive')

# https://stackoverflow.com/questions/52733786/how-to-import-custom-modules-in-google-colab
import sys
sys.path.append('/content/gdrive/MyDrive/SEMRUSH/')

%aimport my

import sys
import numpy as np
import pandas as pd
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import pickle

import my
from my import p

pd.set_option("max_colwidth", 35)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from IPython.display import HTML, display

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
     
 # номер или название эксперимента
# n_exp = '1/'
dir_out = "/content/gdrive/MyDrive/SEMRUSH/out/"
dir_data = '/content/gdrive/MyDrive/SEMRUSH/data/'
os.makedirs(dir_out, exist_ok = True)
RANDOM_STATE = 34

Mounted at /content/gdrive


In [ ]:
# !wget -P /content/gdrive/MyDrive/SEMRUSH/data https://boosters.pro/api/ch/files/pub/semrush_cup_train_data.csv https://boosters.pro/api/ch/files/pub/semrush_cup_categories_and_demo.csv https://boosters.pro/api/ch/files/pub/semrush_cup_test_data.csv https://boosters.pro/api/ch/files/pub/semrush_cup_sub.csv

In [2]:
train = pd.read_csv(dir_data+'train_data.csv',\
                    dtype={'event_group_id':np.int32,
                          'device_id':np.int32 })
train = my.fillna_dom_url(train)
p(train.shape, 13040643/11)
train[:12]

(13040643, 9) 
 ~
1185513.0 
 ~


,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y
0,0,224312,1177617984,tWtXRJ.jl,n,tWAmxZ.tWtXRJ.jl/jdSNxb,n,0,0
1,0,224312,1177618316,ttdXfB.jg,n,tWAmxZ.ttdXfB.jg/jdjizb,n,1,0
2,0,224312,1177618348,ttnqBJ.jg,n,tWAmxZ.ttnqBJ.jg/jdSNxb,n,2,0
3,0,224312,1177618382,ttdXfB.jg,n,tWAmxZ.ttdXfB.jg/jdjizb,n,3,0
4,0,224312,1177618416,tWVsHB.HB,ttnqBJ.jg,tWVsHB.HB/jjKzWr,tWAmxZ.ttnqBJ.jg/jlIdpW/jdSNxb,4,0
5,0,224312,1177618464,tttRWL.jg,n,tWAmxZ.tttRWL.jg/jvzXiI,n,5,0
6,0,224312,1177618470,ttGfsg.jg,n,ttGfsg.jg/jjSKQK,n,6,0
7,0,224312,1177618509,ttXfgB.jg,ttXVAm.jg,ttXfgB.jg/jdSNxb,ttXVAm.jg/jrZgBf,7,0
8,0,224312,1177618512,ttUHGL.AQ,n,ttUHGL.AQ/jrIIbl,n,8,1
9,0,224312,1177618569,ttfLTH.jg,ttUHGL.AQ,ttfLTH.jg/jvzXiI,ttUHGL.AQ/jrIIbl,9,0


In [ ]:
test = pd.read_csv(dir_data+'test_data.csv',\
                    dtype={'event_group_id':np.int32,
                          'device_id':np.int32 })
test = my.fillna_dom_url(test)
p(test.shape)
test[:12]

(6077489, 6) 
 ~


,event_group_id,device_id,timestamp,domain,url,pos
0,1,327782,1168864757,tWtjGv.jg,ttBWVB.tWtjGv.jg/jHzuYQ/jjduQF/...,0
1,1,327782,1168864762,tttTNj.jg,tWJMnu.ttRVWU.tttTNj.jg/jCzlGY,1
2,1,327782,1168864763,ttWZBB.jg,tWAmxZ.ttWZBB.jg/jjKguQ,2
3,1,327782,1168864772,tWMjok.jl,tWAmxZ.tWMjok.jl/jCxBzU/jjbsBr/...,3
4,1,327782,1168864787,ttXxgg.jg,ttXxgg.jg/jCNWvC/jlFIlW,4
5,1,327782,1168864789,ttWqUo.jg,ttWqUo.jg/jdSNxb,5
6,1,327782,1168864798,ttxRzj.jg,tWzdCo.ttxRzj.jg/jCbYCj/jrHuji,6
7,1,327782,1168864800,ttxzmu.jg,ttxzmu.jg/jdLcbc/jrlKHi/jHLXvg,7
8,1,327782,1168864824,ttfZjH.jl,ttgXAW.ttfZjH.jl/jvjiUY,8
9,1,327782,1168864825,ttWAkZ.HM,ttWAkZ.HM/jrsbLN,9


In [ ]:
cats = pd.read_csv(dir_data+'cats.csv')
cats.dropna(subset=['domain'],inplace=True)
cats.category_1.fillna('n',inplace=True)
cats.category_2.fillna('n',inplace=True)
cats.iloc[:,3:]=cats.iloc[:,3:].fillna(0.).astype(np.float32)
p(cats.shape)
cats[:3]

(1229057, 11) 
 ~


,domain,category_1,category_2,M,F,18-24,25-34,35-44,45-54,55-64,65+
0,ttGJTM.AK,n,n,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,ttNggB.jg,n,mass_media,0.600,0.400,0.210,0.480,0.150,0.100,0.040,0.030
2,ttdgZf.jg,n,n,0.550,0.450,0.240,0.300,0.140,0.130,0.090,0.090


In [ ]:
all_domains = pd.concat([cats.domain,train.domain,test.domain,pd.Series(['n'])])\
        .drop_duplicates().to_frame().rename(columns={0:'domain'})
my.split_domain(all_domains,'domain')    
all_domains

,domain,root_domain,ltd_domain
0,ttGJTM.AK,ttGJTM,AK
1,ttNggB.jg,ttNggB,jg
2,ttdgZf.jg,ttdgZf,jg
3,ttsUFm.MZ,ttsUFm,MZ
4,ttBWMn.jg,ttBWMn,jg
...,...,...,...
6456166,ttmqnX.gv,ttmqnX,gv
8067924,ttxNms.jg,ttxNms,jg
8216012,ttAdZG.CA,ttAdZG,CA
5216609,ttxAXq.gS,ttxAXq,gS


In [ ]:
all_urls = pd.concat([train.url,train.url_ref,test.url])\
        .drop_duplicates().to_frame().rename(columns={0:'url'})
my.split_path(all_urls,'url')    
all_urls

,url,path_url
0,tWAmxZ.tWtXRJ.jl/jdSNxb,jdSNxb
1,tWAmxZ.ttdXfB.jg/jdjizb,jdjizb
2,tWAmxZ.ttnqBJ.jg/jdSNxb,jdSNxb
4,tWVsHB.HB/jjKzWr,jjKzWr
5,tWAmxZ.tttRWL.jg/jvzXiI,jvzXiI
...,...,...
6077465,tWzBfU.ttBzRA.jg/jCFfzG/jdSZFg/...,jdbjsF
6077466,ttLBCJ.jg/jCUZWQ,jCUZWQ
6077468,ttRCWg.ttBzRA.jg/jdZgZQ/jlsIKN/...,jrKUjj
6077477,tWxFvz.tWvUNJ.Xv/jCxBzU/jdZFQs/...,jrUzuI


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoders = {}
encoders['domain_enc'] = LabelEncoder().fit(all_domains['domain'])
encoders['root_enc'] = LabelEncoder().fit(all_domains['root_domain'])
encoders['ltd_enc'] = LabelEncoder().fit(all_domains['ltd_domain'])

encoders['url_enc'] = LabelEncoder().fit(all_urls['url'])
encoders['path_enc'] = LabelEncoder().fit(all_urls['path_url'])

encoders['cat_enc'] = LabelEncoder().fit(cats.category_1)

my.save_pickle(dir_data+'encs.pickle', encoders)

In [ ]:
map_cats = [("domain", "domain_enc"), ("category_1", "cat_enc"),
           ("category_2", "cat_enc")]
my.encode_cols(cats, map_cats)
cats.to_parquet(dir_data+'cats.p',index=False)
cats[:5]

,domain,category_1,category_2,M,F,18-24,25-34,35-44,45-54,55-64,65+
0,416064,65,65,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,574675,65,58,0.600,0.400,0.210,0.480,0.150,0.100,0.040,0.030
2,807981,65,65,0.550,0.450,0.240,0.300,0.140,0.130,0.090,0.090
3,1064581,65,65,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,335527,65,65,0.510,0.490,0.160,0.270,0.190,0.140,0.120,0.120


stat

In [3]:
stat = train
my.split_domain(stat, 'domain')
my.split_domain(stat, 'ref')
my.split_path(stat, 'url')
my.split_path(stat, 'url_ref')

map_domain = [("domain", "domain_enc"), ("root_domain", "root_enc"),
              ("ltd_domain", "ltd_enc"), ("url", "url_enc"),
              ("path_url", "path_enc")]

map_ref = [("ref", "domain_enc"), ("root_ref", "root_enc"),
           ("ltd_ref", "ltd_enc"), ("url_ref", "url_enc"),
           ("path_url_ref", "path_enc")]

my.encode_cols(stat, map_domain + map_ref)

stat[:12]

,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y,root_domain,ltd_domain,root_ref,ltd_ref,path_url,path_url_ref
0,0,224312,1177617984,189763,1,531346,1,0,0,157605,233,1,263,1317010,3900817
1,0,224312,1177618316,804359,1,2108910,1,1,0,670068,231,1,263,1531109,3900817
2,0,224312,1177618348,988665,1,2549286,1,2,0,823669,231,1,263,1317010,3900817
3,0,224312,1177618382,804359,1,2108910,1,3,0,670068,231,1,263,1531109,3900817
4,0,224312,1177618416,141092,988665,3538739,2549382,4,0,117227,85,823669,231,1812412,1317010
5,0,224312,1177618464,1092605,1,2789805,1,5,0,910187,231,1,263,3890871,3900817
6,0,224312,1177618470,427466,1,4604601,1,6,0,355515,231,1,263,1867722,3900817
7,0,224312,1177618509,748922,743826,5706576,5690945,7,0,623793,231,619569,231,1317010,3066452
8,0,224312,1177618512,649079,1,5384306,1,8,1,540405,12,1,263,2886127,3900817
9,0,224312,1177618569,826141,649079,6005370,5384306,9,0,688133,231,540405,12,3890871,2886127


In [4]:
stat.shape

(13040643, 15)

In [5]:
my.add_cats_1(stat, ['domain','ref'],dir_data+'cats.p')
stat[:7]

,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y,root_domain,ltd_domain,root_ref,ltd_ref,path_url,path_url_ref,cat_domain,cat_ref
0,0,224312,1177617984,189763,1,531346,1,0,0,157605,233,1,263,1317010,3900817,65,0
1,0,224312,1177618316,804359,1,2108910,1,1,0,670068,231,1,263,1531109,3900817,65,0
2,0,224312,1177618348,988665,1,2549286,1,2,0,823669,231,1,263,1317010,3900817,65,0
3,0,224312,1177618382,804359,1,2108910,1,3,0,670068,231,1,263,1531109,3900817,65,0
4,0,224312,1177618416,141092,988665,3538739,2549382,4,0,117227,85,823669,231,1812412,1317010,65,65
5,0,224312,1177618464,1092605,1,2789805,1,5,0,910187,231,1,263,3890871,3900817,65,0
6,0,224312,1177618470,427466,1,4604601,1,6,0,355515,231,1,263,1867722,3900817,65,0


In [6]:
col_pairs = [("domain", "ref"), ("root_domain", "root_ref"),
             ("ltd_domain", "ltd_ref"), ("cat_domain", "cat_ref"),
             ("url", "url_ref"), ("path_url", "path_url_ref")]

my.make_pairs(stat, col_pairs)
stat[:3]

,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y,root_domain,...,path_url,path_url_ref,cat_domain,cat_ref,pair_domain,pair_root_domain,pair_ltd_domain,pair_cat_domain,pair_url,pair_path_url
0,0,224312,1177617984,189763,1,531346,1,0,0,157605,...,1317010,3900817,65,0,1897630000001,1576050000001,2330000263,650000000,5313460000001,13170103900817
1,0,224312,1177618316,804359,1,2108910,1,1,0,670068,...,1531109,3900817,65,0,8043590000001,6700680000001,2310000263,650000000,21089100000001,15311093900817
2,0,224312,1177618348,988665,1,2549286,1,2,0,823669,...,1317010,3900817,65,0,9886650000001,8236690000001,2310000263,650000000,25492860000001,13170103900817


In [7]:
stat[:12]

,event_group_id,device_id,timestamp,domain,ref,url,url_ref,pos,y,root_domain,...,path_url,path_url_ref,cat_domain,cat_ref,pair_domain,pair_root_domain,pair_ltd_domain,pair_cat_domain,pair_url,pair_path_url
0,0,224312,1177617984,189763,1,531346,1,0,0,157605,...,1317010,3900817,65,0,1897630000001,1576050000001,2330000263,650000000,5313460000001,13170103900817
1,0,224312,1177618316,804359,1,2108910,1,1,0,670068,...,1531109,3900817,65,0,8043590000001,6700680000001,2310000263,650000000,21089100000001,15311093900817
2,0,224312,1177618348,988665,1,2549286,1,2,0,823669,...,1317010,3900817,65,0,9886650000001,8236690000001,2310000263,650000000,25492860000001,13170103900817
3,0,224312,1177618382,804359,1,2108910,1,3,0,670068,...,1531109,3900817,65,0,8043590000001,6700680000001,2310000263,650000000,21089100000001,15311093900817
4,0,224312,1177618416,141092,988665,3538739,2549382,4,0,117227,...,1812412,1317010,65,65,1410920988665,1172270823669,850000231,650000065,35387392549382,18124121317010
5,0,224312,1177618464,1092605,1,2789805,1,5,0,910187,...,3890871,3900817,65,0,10926050000001,9101870000001,2310000263,650000000,27898050000001,38908713900817
6,0,224312,1177618470,427466,1,4604601,1,6,0,355515,...,1867722,3900817,65,0,4274660000001,3555150000001,2310000263,650000000,46046010000001,18677223900817
7,0,224312,1177618509,748922,743826,5706576,5690945,7,0,623793,...,1317010,3066452,65,65,7489220743826,6237930619569,2310000231,650000065,57065765690945,13170103066452
8,0,224312,1177618512,649079,1,5384306,1,8,1,540405,...,2886127,3900817,65,0,6490790000001,5404050000001,120000263,650000000,53843060000001,28861273900817
9,0,224312,1177618569,826141,649079,6005370,5384306,9,0,688133,...,3890871,2886127,65,65,8261410649079,6881330540405,2310000012,650000065,60053705384306,38908712886127


In [ ]:
stat.dtypes

event_group_id      int32
device_id           int32
timestamp           int64
domain              int32
ref                 int32
url                 int32
url_ref             int32
pos                  int8
y                    int8
root_domain         int32
ltd_domain          int16
root_ref            int32
ltd_ref             int16
path_url            int32
path_url_ref        int32
cat_domain          int32
cat_ref             int32
pair_domain         int64
pair_root_domain    int64
pair_ltd_domain     int64
pair_cat_domain     int64
pair_url            int64
pair_path_url       int64
dtype: object

In [ ]:
stat.to_parquet('stat.p',index=False)
del stat
gc.collect()

train

In [2]:
train = pd.read_csv(dir_data+'train_data.csv',\
                    dtype={'event_group_id':np.int32,
                          'device_id':np.int32 })

test = pd.read_csv(dir_data+'test_data.csv',\
                    dtype={'event_group_id':np.int32,
                          'device_id':np.int32 })

train = my.fillna_dom_url(train).drop(columns=['ref','url_ref'])
test = my.fillna_dom_url(test)
display(train[:3])
display(test[:3])

,event_group_id,device_id,timestamp,domain,url,pos,y
0,0,224312,1177617984,tWtXRJ.jl,tWAmxZ.tWtXRJ.jl/jdSNxb,0,0
1,0,224312,1177618316,ttdXfB.jg,tWAmxZ.ttdXfB.jg/jdjizb,1,0
2,0,224312,1177618348,ttnqBJ.jg,tWAmxZ.ttnqBJ.jg/jdSNxb,2,0


,event_group_id,device_id,timestamp,domain,url,pos
0,1,327782,1168864757,tWtjGv.jg,ttBWVB.tWtjGv.jg/jHzuYQ/jjduQF/...,0
1,1,327782,1168864762,tttTNj.jg,tWJMnu.ttRVWU.tttTNj.jg/jCzlGY,1
2,1,327782,1168864763,ttWZBB.jg,tWAmxZ.ttWZBB.jg/jjKguQ,2


In [3]:
my.count_parts_url(train)
my.count_parts_url(test)
train[:3]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain
0,0,224312,1177617984,tWtXRJ.jl,tWAmxZ.tWtXRJ.jl/jdSNxb,0,0,1,2
1,0,224312,1177618316,ttdXfB.jg,tWAmxZ.ttdXfB.jg/jdjizb,1,0,1,2
2,0,224312,1177618348,ttnqBJ.jg,tWAmxZ.ttnqBJ.jg/jdSNxb,2,0,1,2


In [4]:
def f1(t):
  my.split_domain(t, 'domain')
  my.split_path(t, 'url')

  map_domain = [("domain", "domain_enc"), ("root_domain", "root_enc"),
              ("ltd_domain", "ltd_enc"), ("url", "url_enc"),
              ("path_url", "path_enc")]

  my.encode_cols(t, map_domain)

f1(train)
f1(test)
train[:3]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,root_domain,ltd_domain,path_url
0,0,224312,1177617984,189763,531346,0,0,1,2,157605,233,1317010
1,0,224312,1177618316,804359,2108910,1,0,1,2,670068,231,1531109
2,0,224312,1177618348,988665,2549286,2,0,1,2,823669,231,1317010


In [5]:
def add_targets(t):
  pos_target=10
  tar_cols = ['event_group_id','domain','root_domain','ltd_domain',\
              'url','path_url','ff_count_slash','ff_count_subdomain']
  
  targets = t.loc[t.pos==pos_target, tar_cols]\
        .set_index('event_group_id')
  targets.columns = ['target', 'root_target', 'ltd_target',
       'url_target', 'path_url_target', 'ff_count_slash_target',\
       'ff_count_subdomain_target']
  t = t.merge(targets,left_on='event_group_id',right_index=True)
  t.sort_values(['event_group_id', 'pos'], inplace=True)
  return t

In [6]:
train = add_targets(train)
test = add_targets(test)
train[:15]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,root_domain,ltd_domain,path_url,target,root_target,ltd_target,url_target,path_url_target,ff_count_slash_target,ff_count_subdomain_target
0,0,224312,1177617984,189763,531346,0,0,1,2,157605,233,1317010,621856,517644,231,1659176,1521773,2,2
1,0,224312,1177618316,804359,2108910,1,0,1,2,670068,231,1531109,621856,517644,231,1659176,1521773,2,2
2,0,224312,1177618348,988665,2549286,2,0,1,2,823669,231,1317010,621856,517644,231,1659176,1521773,2,2
3,0,224312,1177618382,804359,2108910,3,0,1,2,670068,231,1531109,621856,517644,231,1659176,1521773,2,2
4,0,224312,1177618416,141092,3538739,4,0,1,1,117227,85,1812412,621856,517644,231,1659176,1521773,2,2
5,0,224312,1177618464,1092605,2789805,5,0,1,2,910187,231,3890871,621856,517644,231,1659176,1521773,2,2
6,0,224312,1177618470,427466,4604601,6,0,1,1,355515,231,1867722,621856,517644,231,1659176,1521773,2,2
7,0,224312,1177618509,748922,5706576,7,0,1,1,623793,231,1317010,621856,517644,231,1659176,1521773,2,2
8,0,224312,1177618512,649079,5384306,8,1,1,1,540405,12,2886127,621856,517644,231,1659176,1521773,2,2
9,0,224312,1177618569,826141,6005370,9,0,1,1,688133,231,3890871,621856,517644,231,1659176,1521773,2,2


In [7]:
train.target.nunique(), train.loc[train.pos==10,'domain'].nunique()

(163932, 163932)

In [8]:
my.add_cats_1(train, ['domain','target'],dir_data+'cats.p')
my.add_cats_1(test, ['domain','target'],dir_data+'cats.p')
train[:3]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,root_domain,...,path_url,target,root_target,ltd_target,url_target,path_url_target,ff_count_slash_target,ff_count_subdomain_target,cat_domain,cat_target
0,0,224312,1177617984,189763,531346,0,0,1,2,157605,...,1317010,621856,517644,231,1659176,1521773,2,2,65,65
1,0,224312,1177618316,804359,2108910,1,0,1,2,670068,...,1531109,621856,517644,231,1659176,1521773,2,2,65,65
2,0,224312,1177618348,988665,2549286,2,0,1,2,823669,...,1317010,621856,517644,231,1659176,1521773,2,2,65,65


In [9]:
col_pairs = [("target", "domain"),
             ("root_target", "root_domain"),
             ("ltd_target", "ltd_domain"),
            ("cat_target", "cat_domain"),
             ("url_target", "url"), 
             ("path_url_target", "path_url")]

my.make_pairs(train, col_pairs)
my.make_pairs(test, col_pairs)
train[:3]

,event_group_id,device_id,timestamp,domain,url,pos,y,ff_count_slash,ff_count_subdomain,root_domain,...,ff_count_slash_target,ff_count_subdomain_target,cat_domain,cat_target,pair_target,pair_root_target,pair_ltd_target,pair_cat_target,pair_url_target,pair_path_url_target
0,0,224312,1177617984,189763,531346,0,0,1,2,157605,...,2,2,65,65,6218560189763,5176440157605,2310000233,650000065,16591760531346,15217731317010
1,0,224312,1177618316,804359,2108910,1,0,1,2,670068,...,2,2,65,65,6218560804359,5176440670068,2310000231,650000065,16591762108910,15217731531109
2,0,224312,1177618348,988665,2549286,2,0,1,2,823669,...,2,2,65,65,6218560988665,5176440823669,2310000231,650000065,16591762549286,15217731317010


In [10]:
train.columns

Index(['event_group_id', 'device_id', 'timestamp', 'domain', 'url', 'pos', 'y',
       'ff_count_slash', 'ff_count_subdomain', 'root_domain', 'ltd_domain',
       'path_url', 'target', 'root_target', 'ltd_target', 'url_target',
       'path_url_target', 'ff_count_slash_target', 'ff_count_subdomain_target',
       'cat_domain', 'cat_target', 'pair_target', 'pair_root_target',
       'pair_ltd_target', 'pair_cat_target', 'pair_url_target',
       'pair_path_url_target'],
      dtype='object')

In [11]:
train.to_parquet(dir_data+'train_data.p',index=False)
test.to_parquet(dir_data+'test_data.p',index=False)
p(train.isna().sum().sum(),test.isna().sum().sum())

0 
 ~
0 
 ~


In [12]:
test[:3]

,event_group_id,device_id,timestamp,domain,url,pos,ff_count_slash,ff_count_subdomain,root_domain,ltd_domain,...,ff_count_slash_target,ff_count_subdomain_target,cat_domain,cat_target,pair_target,pair_root_target,pair_ltd_target,pair_cat_target,pair_url_target,pair_path_url_target
0,1,327782,1168864757,194254,4242956,0,5,2,161321,231,...,2,2,65,65,11805900194254,9837450161321,2310000231,650000065,40397604242956,28808603253601
1,1,327782,1168864762,1093395,3141193,1,1,3,910835,231,...,2,2,65,65,11805901093395,9837450910835,2310000231,650000065,40397603141193,28808600547807
2,1,327782,1168864763,717284,1881192,2,1,2,597474,231,...,2,2,65,65,11805900717284,9837450597474,2310000231,650000065,40397601881192,28808601806665


In [13]:
test.columns

Index(['event_group_id', 'device_id', 'timestamp', 'domain', 'url', 'pos',
       'ff_count_slash', 'ff_count_subdomain', 'root_domain', 'ltd_domain',
       'path_url', 'target', 'root_target', 'ltd_target', 'url_target',
       'path_url_target', 'ff_count_slash_target', 'ff_count_subdomain_target',
       'cat_domain', 'cat_target', 'pair_target', 'pair_root_target',
       'pair_ltd_target', 'pair_cat_target', 'pair_url_target',
       'pair_path_url_target'],
      dtype='object')